In [ ]:
import os
import asyncio
import aiohttp
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm
from dotenv import load_dotenv
from ratelimit import limits, sleep_and_retry

# Load environment variables
load_dotenv()
api_key = os.getenv('api')

# Database connection
engine = create_engine('mssql+pyodbc://Revision-PC\\SQLEXPRESS/RiotDataDB?driver=ODBC+Driver+17+for+SQL+Server')

# Rate limit parameters
RATE_LIMIT = 100  # 100 requests per minute
TIME_WINDOW = 60  # 60 seconds

# Decorator to handle rate limiting
@sleep_and_retry
@limits(calls=RATE_LIMIT, period=TIME_WINDOW)
async def fetch_match_ids(session, puuid):
    api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&api_key={api_key}"
    async with session.get(api_url) as response:
        response.raise_for_status()
        return await response.json()

async def fetch_match_detail(session, match_id):
    match_details_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
    async with session.get(match_details_url) as response:
        response.raise_for_status()
        match_details_json = await response.json()
        return match_details_json

async def process_puuid(session, puuid):
    match_ids = await fetch_match_ids(session, puuid)
    match_details_list = []
    for match_id in match_ids:
        match_details_json = await fetch_match_detail(session, match_id)
        queue_id = match_details_json.get('info', {}).get('queueId', None)
        if queue_id == 420:  # Only ranked solo queue
            match_details_list.append({"puuid": puuid, "match_id": match_id, "queue_id": queue_id})
    return match_details_list

async def main():
    # Read PUUIDs from the database
    df_puuid = pd.read_sql_query("SELECT puuid FROM SummID_PUUID", engine)

    # Initialize an empty list to collect match data
    match_id_list = []

    async with aiohttp.ClientSession() as session:
        tasks = [process_puuid(session, puuid) for puuid in df_puuid['puuid']]
        for future in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Processing PUUUIDs", unit="puuid"):
            result = await future
            match_id_list.extend(result)

    # Convert the list to a DataFrame and save to the database
    df_match_ids = pd.DataFrame(match_id_list)
    df_match_ids.to_sql('MatchID', con=engine, if_exists='replace', index=False)
    print("Match IDs and details saved successfully.")

if __name__ == "__main__":
    asyncio.run(main())
